In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='/content/data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='/content/data/', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

0it [00:00, ?it/s]

9920512it [00:01, 8759729.40it/s]                            


Extracting /content/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 136249.36it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/data/MNIST/raw


1654784it [00:00, 2205547.13it/s]                            
0it [00:00, ?it/s]

Extracting /content/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/MNIST/raw


8192it [00:00, 49960.94it/s]            


Extracting /content/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/MNIST/raw
Processing...
Done!


In [ ]:
# for batch_idx, (data, _) in enumerate(train_loader):
#     x= data

In [ ]:
# save_image(data[1].view(1, 1, 28, 28), '/content/sample_1' + '.png')

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim, h1_dim, h2_dim, z_dim):
    super(VAE, self).__init__()
    
    #encoder layer
    self.e_layer1 = nn.Linear(x_dim, h1_dim)
    self.e_layer2 = nn.Linear(h1_dim, h2_dim)
    
    self.mu = nn.Linear(h2_dim, z_dim)
    self.log_var = nn.Linear(h2_dim, z_dim)
    
    #decoder layer
    self.d_layer1 = nn.Linear(z_dim, h2_dim)
    self.d_layer2 = nn.Linear(h2_dim, h1_dim)
    self.d_layer3 = nn.Linear(h1_dim, x_dim)
    
  def encoder(self, x):
    h = F.relu(self.e_layer1(x))
    h = F.relu(self.e_layer2(h))
    mu = self.mu(h)
    log_var = self.log_var(h)
    return mu, log_var
  
  def sampling(self, mu, log_var):
    var = torch.exp(0.5*log_var)
    eps = torch.randn_like(var)
    return eps.mul(var).add_(mu)
  
  def decoder(self, z):
    h = F.relu(self.d_layer1(z))
    h = F.relu(self.d_layer2(h))
    output = F.sigmoid(self.d_layer3(h))
    return output
  
  def forward(self, input):
    mu, log_var = self.encoder(input.view(-1,784))
    z = self.sampling(mu, log_var)
    output = self.decoder(z)
    return output, mu, log_var



In [ ]:
vae = VAE(784, 512, 256, 2)
# if torch.cuda.is_available():
#     vae.cuda()
    

In [ ]:
vae

VAE(
  (e_layer1): Linear(in_features=784, out_features=512, bias=True)
  (e_layer2): Linear(in_features=512, out_features=256, bias=True)
  (mu): Linear(in_features=256, out_features=2, bias=True)
  (log_var): Linear(in_features=256, out_features=2, bias=True)
  (d_layer1): Linear(in_features=2, out_features=256, bias=True)
  (d_layer2): Linear(in_features=256, out_features=512, bias=True)
  (d_layer3): Linear(in_features=512, out_features=784, bias=True)
)

In [ ]:
def loss_function(recon_x, x, mu, log_var):
  KL = -0.5*torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
  CE = F.binary_cross_entropy(recon_x, x.view(-1,784), reduction = 'sum')
  return KL + CE


In [ ]:
optimizer = optim.Adam(vae.parameters())

In [ ]:
def train(epoches):
  vae.train()
  train_loss = 0
  for batch_idx, (data, _) in enumerate(train_loader):
    
#     if torch.cuda.is_available():
#       data.cuda()
    
    optimizer.zero_grad()
    
    recon_x, mu, log_var = vae(data)
    loss = loss_function(recon_x, data, mu, log_var)
    
    loss.backward()
    train_loss += loss.item()
    optimizer.step()
    
    if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
  print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
    

In [ ]:
def test():
  vae.eval()
  test_loss = 0
  with torch.no_grad():
    for data, _ in test_loader:
#       if torch.cuda.is_available():
#         data.cuda()
      
      recon_data, mu, log_var = vae(data)
      loss = loss_function(recon_data, data, mu, log_var)
      
      test_loss += loss
      
  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))      
    

In [ ]:
for epoch in range(1, 51):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 545.237891
Train Epoch: 1 [10000/60000 (17%)]	Loss: 193.617324
Train Epoch: 1 [20000/60000 (33%)]	Loss: 171.796484
Train Epoch: 1 [30000/60000 (50%)]	Loss: 174.670703
Train Epoch: 1 [40000/60000 (67%)]	Loss: 170.208633
Train Epoch: 1 [50000/60000 (83%)]	Loss: 164.590840
====> Epoch: 1 Average loss: 179.0982
====> Test set loss: 162.1939
Train Epoch: 2 [0/60000 (0%)]	Loss: 156.130908
Train Epoch: 2 [10000/60000 (17%)]	Loss: 157.924141
Train Epoch: 2 [20000/60000 (33%)]	Loss: 159.876729
Train Epoch: 2 [30000/60000 (50%)]	Loss: 151.808057
Train Epoch: 2 [40000/60000 (67%)]	Loss: 158.874111
Train Epoch: 2 [50000/60000 (83%)]	Loss: 158.268740
====> Epoch: 2 Average loss: 157.7264
====> Test set loss: 154.8526
Train Epoch: 3 [0/60000 (0%)]	Loss: 161.476729
Train Epoch: 3 [10000/60000 (17%)]	Loss: 160.317002
Train Epoch: 3 [20000/60000 (33%)]	Loss: 157.707656
Train Epoch: 3 [30000/60000 (50%)]	Loss: 147.490967
Train Epoch: 3 [40000/60000 (67%)]	Loss: 159.67

In [ ]:

with torch.no_grad():
    z = torch.randn(64, 2)
    print(z)
    sample = vae.decoder(z)
    save_image(sample.view(64, 1, 28, 28), 'drive/My Drive/activelaerning/homework2/sample_image1' + '.png')
    

tensor([[-1.8327,  0.6776],
        [ 0.1008,  0.7682],
        [-0.9421, -1.0095],
        [-0.0610, -1.1693],
        [ 0.1354, -1.2014],
        [-0.8841,  0.0337],
        [ 2.2786,  2.2434],
        [-0.2016, -0.5290],
        [-0.0047, -0.0970],
        [-0.0092,  0.2998],
        [ 1.2681,  0.2654],
        [ 0.8999,  0.1292],
        [ 1.9096,  0.4173],
        [ 0.7496, -1.5915],
        [-0.1208, -0.4132],
        [-0.3170, -0.0283],
        [-0.2089,  1.1713],
        [ 0.4444, -0.3043],
        [ 1.5079,  0.6345],
        [ 0.2383,  1.8588],
        [ 0.2820, -0.0825],
        [ 0.8087, -0.7634],
        [-1.4636, -0.2152],
        [-1.7808,  1.3852],
        [ 0.4493, -1.3441],
        [ 0.5636, -0.8082],
        [ 1.4323, -1.9749],
        [ 0.2321, -0.0047],
        [-0.2981, -0.3689],
        [-1.1723, -0.3658],
        [ 0.1300, -0.9767],
        [ 0.3471,  0.2877],
        [ 1.8667,  1.0478],
        [-3.4196, -1.8831],
        [-0.5876,  0.4294],
        [ 1.6508, -0

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
torch.save(vae,"drive/My Drive/activelaerning/homework2/model.h5")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
the_model = torch.load("drive/My Drive/activelaerning/homework2/model.h5")


In [ ]:

with torch.no_grad():
    z = torch.randn(64, 2)
#     print(z)
    sample = the_model.decoder(z)
    save_image(sample.view(64, 1, 28, 28), 'sample_image1' + '.png')
    

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
